In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys


import xgboost as xgb
from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')
sys.path.append('../src')
from irish_buoy_data import IrishBuoyData

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader

# Preparing the Data Frame

In [ ]:
# Step 1: Get your multi-level data (you already have this)
buoy_stations = ['M2', 'M3', 'M4', 'M5', 'M6']
all_buoy_data = []

for station_id in buoy_stations:
    try:
        buoy = IrishBuoyData(station_id=station_id)
        data = buoy.fetch_data(days_back=1875)
        data = data[~data.index.duplicated(keep='first')]
        data = data.drop(columns=['station_id'])
        all_buoy_data.append(data)
    except Exception as e:
        print(f"  ✗ {station_id}: Error - {e}")

# Create multi-level DataFrame
df_multi = pd.concat(all_buoy_data, axis=1, keys=buoy_stations)
df_multi = df_multi.interpolate(method='time', limit=3).fillna(method='ffill').fillna(method='bfill')

# Step 2: Convert to 3D tensor [time, buoys, features]
def multi_to_tensor(df_multi):
    """
    Convert multi-level DataFrame to 3D tensor
    Structure: [timesteps, buoys, features]
    """
    buoy_ids = df_multi.columns.levels[0].tolist()
    n_timesteps = len(df_multi)
    n_buoys = len(buoy_ids)
    n_features = len(df_multi[buoy_ids[0]].columns)
    
    # Initialize tensor
    X_tensor = np.zeros((n_timesteps, n_buoys, n_features))
    
    # Fill tensor
    for b_idx, buoy in enumerate(buoy_ids):
        X_tensor[:, b_idx, :] = df_multi[buoy].values
    
    return X_tensor

# Create the tensor
X_tensor = multi_to_tensor(df_multi)

print(f"✓ Tensor created!")
print(f"Shape: {X_tensor.shape}")
print(f"  - Timesteps: {X_tensor.shape[0]}")
print(f"  - Buoys: {X_tensor.shape[1]}")
print(f"  - Features: {X_tensor.shape[2]}")

# Verify structure
print(f"\n=== Verification ===")
print(f"Feature names: {df_multi['M2'].columns.tolist()}")
print(f"\nM3 state at first timestep:")
print(X_tensor[0, 2, :])  # buoy index 2 = M3

print(f"\nDataFrame equivalent:")
print(df_multi['M4'].iloc[0])

✓ Tensor created!
Shape: (44201, 5, 6)
  - Timesteps: 44201
  - Buoys: 5
  - Features: 6

=== Verification ===
Feature names: ['WindSpeed (knots)', 'AirTemperature (degrees_C)', 'AtmosphericPressure (millibars)', 'WaveHeight (meters)', 'Hmax (meters)', 'Tp (seconds)']

M3 state at first timestep:
[  22.656   11.387 1010.828    2.656    3.633   12.188]

DataFrame equivalent:
WindSpeed (knots)                    22.656
AirTemperature (degrees_C)           11.387
AtmosphericPressure (millibars)    1010.828
WaveHeight (meters)                   2.656
Hmax (meters)                         3.633
Tp (seconds)                         12.188
Name: 2020-11-10 14:00:00+00:00, dtype: float64


/var/folders/1j/0q6grdh110z1c008s2nrczkm0000gn/T/ipykernel_44847/4109033216.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_multi = df_multi.interpolate(method='time', limit=3).fillna(method='ffill').fillna(method='bfill')


# Preparing the Model

### STEP 1: Prepare Data with External Forcing U_t


In [ ]:
# ============================================================================
# STEP 1: Prepare Data with External Forcing U_t
# ============================================================================

def compute_external_forcing(df_multi):
    """
    Compute external forcing U_t:
    - P(t): Mean atmospheric pressure across network
    - ∇P(t): Pressure gradient (spatial derivative)
    - W(t): Mean wind speed
    - W_dir(t): Wind direction proxy
    """
    buoy_ids = ['M2', 'M3', 'M4', 'M5', 'M6']
    
    # Extract pressure and wind for all buoys
    pressures = np.array([df_multi[buoy]['AtmosphericPressure (millibars)'].values 
                          for buoy in buoy_ids])  # (n_buoys, timesteps)
    winds = np.array([df_multi[buoy]['WindSpeed (knots)'].values 
                      for buoy in buoy_ids])
    
    # Compute forcing components
    P_t = pressures.mean(axis=0)  # Mean pressure
    div_P_t = pressures.std(axis=0)  # Pressure gradient proxy (std across buoys)
    W_t = winds.mean(axis=0)  # Mean wind speed
    
    # Wind direction proxy: variance in wind speeds across buoys
    W_dir_t = winds.std(axis=0)
    
    # Stack into U_t: (timesteps, 4)
    U_t = np.stack([P_t, div_P_t, W_t, W_dir_t], axis=1)
    
    return U_t


### STEP 2: Create Dataset for Sequence Prediction


In [ ]:
# ============================================================================
# STEP 2: Create Dataset for Sequence Prediction
# ============================================================================

class BuoyForecastDataset(Dataset):
    """
    Dataset for learning F: X_{t+1} = F(X_t, X_{t-1}, U_t) + ε_t
    """
    def __init__(self, X_tensor, U_forcing, sequence_length=2):
        """
        Args:
            X_tensor: (timesteps, n_buoys, n_features)
            U_forcing: (timesteps, n_forcing_vars)
            sequence_length: how many past states to use (default 2 for X_t, X_{t-1})
        """
        self.X = torch.FloatTensor(X_tensor)
        self.U = torch.FloatTensor(U_forcing)
        self.seq_len = sequence_length
        
        # Precompute ΔX (prediction increments)
        self.delta_X = self.X[1:] - self.X[:-1]  # X_{t+1} - X_t
        
    def __len__(self):
        return len(self.X) - self.seq_len
    
    def __getitem__(self, idx):
        # Input: [X_{t-1}, X_t] and U_t
        X_history = self.X[idx:idx+self.seq_len]  # (seq_len, n_buoys, n_features)
        U_current = self.U[idx+self.seq_len-1]    # (n_forcing_vars,)
        
        # Target: ΔX_{t+1} = X_{t+1} - X_t
        target_delta = self.delta_X[idx+self.seq_len-1]  # (n_buoys, n_features)
        
        # Also return X_t for computing X_{t+1} = X_t + ΔX_{t+1}
        X_current = self.X[idx+self.seq_len-1]
        
        return X_history, U_current, target_delta, X_current
    


### STEP 3: Define Neural Network Architecture for F


In [ ]:
# ============================================================================
# STEP 3: Define Neural Network Architecture for F
# ============================================================================

class PhysicsInformedBuoyModel(nn.Module):
    """
    Learn operator F: X_{t+1} = F(X_t, X_{t-1}, U_t) + ε_t
    
    Architecture:
    1. Encode X_t, X_{t-1} with LSTM/GRU
    2. Encode U_t with FC layers
    3. Combine and predict ΔX_{t+1}
    4. Add X_t to get X_{t+1}
    """
    def __init__(self, n_buoys=5, n_features=6, n_forcing=4, hidden_dim=128):
        super().__init__()
        
        self.n_buoys = n_buoys
        self.n_features = n_features
        
        # Temporal encoder for X_t, X_{t-1} (handles sequence)
        self.temporal_encoder = nn.LSTM(
            input_size=n_buoys * n_features,  # Flatten buoys × features
            hidden_size=hidden_dim,
            num_layers=2,
            batch_first=True,
            dropout=0.2
        )
        
        # External forcing encoder for U_t
        self.forcing_encoder = nn.Sequential(
            nn.Linear(n_forcing, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 64),
            nn.ReLU()
        )
        
        # Fusion layer: combine temporal + forcing
        self.fusion = nn.Sequential(
            nn.Linear(hidden_dim + 64, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU()
        )
        
        # Output layer: predict ΔX_{t+1}
        self.output = nn.Linear(128, n_buoys * n_features)
        
    def forward(self, X_history, U_current):
        """
        Args:
            X_history: (batch, seq_len, n_buoys, n_features)
            U_current: (batch, n_forcing)
        Returns:
            delta_X: (batch, n_buoys, n_features) - predicted increment
        """
        batch_size, seq_len, n_buoys, n_features = X_history.shape
        
        # Flatten spatial dimensions for LSTM
        X_flat = X_history.reshape(batch_size, seq_len, -1)  # (batch, seq_len, n_buoys*n_features)
        
        # Encode temporal dynamics
        lstm_out, (h_n, c_n) = self.temporal_encoder(X_flat)
        temporal_features = h_n[-1]  # (batch, hidden_dim)
        
        # Encode external forcing
        forcing_features = self.forcing_encoder(U_current)  # (batch, 64)
        
        # Fuse features
        combined = torch.cat([temporal_features, forcing_features], dim=1)
        fused = self.fusion(combined)
        
        # Predict ΔX
        delta_X_flat = self.output(fused)  # (batch, n_buoys*n_features)
        delta_X = delta_X_flat.reshape(batch_size, n_buoys, n_features)
        
        return delta_X


### STEP 4: Training Loop


In [ ]:
# ============================================================================
# STEP 4: Training Loop
# ============================================================================

def train_model(model, train_loader, val_loader, num_epochs=50, lr=0.001, device='cpu'):
    """
    Train the physics-informed model
    """
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)
    
    # Loss: MSE on increments (captures physics better than direct prediction)
    criterion = nn.MSELoss()
    
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0.0
        
        for X_history, U_current, target_delta, X_current in train_loader:
            X_history = X_history.to(device)
            U_current = U_current.to(device)
            target_delta = target_delta.to(device)
            
            optimizer.zero_grad()
            
            # Predict ΔX_{t+1}
            pred_delta = model(X_history, U_current)
            
            # Loss on increments
            loss = criterion(pred_delta, target_delta)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        
        # Validation
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for X_history, U_current, target_delta, X_current in val_loader:
                X_history = X_history.to(device)
                U_current = U_current.to(device)
                target_delta = target_delta.to(device)
                
                pred_delta = model(X_history, U_current)
                loss = criterion(pred_delta, target_delta)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        
        scheduler.step(val_loss)
        
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
    return train_losses, val_losses


### STEP 5: Multi-Step Forecasting (Autoregressive)

In [ ]:
# ============================================================================
# STEP 5: Multi-Step Forecasting (Autoregressive)
# ============================================================================

def multi_step_forecast(model, X_initial, U_forcing, n_steps, device='cpu'):
    """
    Forecast n_steps ahead using model autoregressively
    
    X_{t+1} = X_t + F(X_t, X_{t-1}, U_t)
    """
    model.eval()
    
    # Initialize with first two states
    X_history = X_initial.unsqueeze(0).to(device)  # (1, seq_len, n_buoys, n_features)
    predictions = [X_history[0, -1].cpu().numpy()]  # Start with X_t
    
    with torch.no_grad():
        for step in range(n_steps):
            # Get current forcing
            U_current = U_forcing[step].unsqueeze(0).to(device)
            
            # Predict ΔX
            delta_X = model(X_history, U_current)
            
            # Compute X_{t+1} = X_t + ΔX
            X_next = X_history[0, -1] + delta_X[0]
            predictions.append(X_next.cpu().numpy())
            
            # Update history: shift and add new prediction
            X_history = torch.cat([X_history[:, 1:], X_next.unsqueeze(0).unsqueeze(0)], dim=1)
    
    return np.array(predictions)



### STEP 6: Main Execution

In [ ]:
# ============================================================================
# STEP 6: Main Execution
# ============================================================================

# Compute external forcing
print("Computing external forcing U_t...")
U_forcing = compute_external_forcing(df_multi)
print(f"U_forcing shape: {U_forcing.shape}")

# Create dataset
print("\nCreating dataset...")
dataset = BuoyForecastDataset(X_tensor, U_forcing, sequence_length=2)
print(f"Dataset size: {len(dataset)}")

# Train/val split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Initialize model
print("\nInitializing model...")
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
model = PhysicsInformedBuoyModel(n_buoys=5, n_features=6, n_forcing=4, hidden_dim=128)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

# Train
print("\nTraining...")
train_losses, val_losses = train_model(model, train_loader, val_loader, num_epochs=50, device=device)

# Save model
torch.save(model.state_dict(), 'physics_informed_buoy_model.pth')
print("\n✓ Model saved!")

# Example forecast
print("\nGenerating 24-hour forecast...")
X_initial = torch.FloatTensor(X_tensor[-2:])  # Last 2 timesteps
U_future = torch.FloatTensor(U_forcing[-24:])  # Next 24 hours of forcing

forecast = multi_step_forecast(model, X_initial, U_future, n_steps=24, device=device)
print(f"Forecast shape: {forecast.shape}")  # (24, 5 buoys, 6 features)

Computing external forcing U_t...
U_forcing shape: (44201, 4)

Creating dataset...
